# Import package

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install Sastrawi
import re
import nltk 
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
import string


In [ ]:
nltk.download()

# Read data

In [ ]:
ig_con=pd.read_excel('/content/drive/MyDrive/satria_data_penyisihan_2/BPJS_Instagram_2022-2021.xlsx')


# Data Preproccesing

## Preprocessing umum

In [ ]:
ig_con

In [ ]:
ig_con['tahun']=ig_con['Waktu_Post'].dt.year
ig_con['month']=ig_con['Waktu_Post'].dt.month

In [ ]:
ig_con['tahun'].value_counts()

## Text preprocessing

### Cleaning data 

In [ ]:
#clear comment yang sama
ig_c=ig_con.copy()
ig_c['Komentar']=ig_c['Komentar'].apply(lambda x:  re.sub(re.compile(r'\s+'), '', x))
ig_c['Konteks']=ig_c['Konteks'].apply(lambda x:  re.sub(re.compile(r'\s+'), '', x))
ig_c=ig_c.reset_index()
idx_sam=ig_c[ig_c['Konteks']!=ig_c['Komentar']]['index']
ig_con=ig_con.iloc[idx_sam]

In [ ]:
def clean(x):
  x=x.lower()
  x=x.strip()
  x=x.translate(str.maketrans('', '', string.punctuation))
  return x

In [ ]:
ig_con['Komentar']=ig_con['Komentar'].apply(clean)
ig_con['Konteks']=ig_con['Konteks'].apply(clean)


In [ ]:
listStopword=list(stopwords.words('indonesian'))
listStopword.extend(['yg','jg','teh','mah','da','atuh','jd','km','ak','lg','ya','ga','ngga','nggak','gak','tp',
                   'kalo','nya','pake','liat','udh','aja','wkwk','wkwkwk','wk','gt','gais','blm','sih','tau',
                   'tahu','gt','udah','utk','rb','rp','dgn','ayo','isi','biar','yah','dr','bawa','gitu','eh',
                   'pas','td','sm','pengen','pgn','dpt','sd','byr','min','dscn','sy','no','sok'])

In [ ]:
def remove_stopwords(sentence):
  words = sentence.split()
  words = [word for word in words if word not in listStopword and word.isalpha()]
  return " ".join(words)

In [ ]:
ig_con['Komentar']=ig_con['Komentar'].apply(remove_stopwords)


In [ ]:
ig_con.to_csv('/content/drive/MyDrive/satria_data_penyisihan_2/bpjs_ig_stem.csv', index = False)

# Sentiment analysis

https://huggingface.co/ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa?text=Saya+mengapresiasi+usaha+anda

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

pretrained_name = "ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa"
nlp = pipeline("sentiment-analysis", model=pretrained_name, tokenizer=pretrained_name)

In [ ]:
ig_con['Komentar']=ig_con['Komentar'].astype('str')

In [ ]:
ig_con.to_csv('/content/drive/MyDrive/satria_data_penyisihan_2/ig_withoutstemming.csv', index = False)

In [ ]:
ig=pd.read_csv('/content/drive/MyDrive/satria_data_penyisihan_2/ig_withoutstemming.csv')

In [ ]:
ig.dropna(inplace=True)

In [ ]:
ig

In [ ]:
ig['label_score'] = ig['Komentar'].apply(lambda value: nlp(value)[0])


In [ ]:
ig

In [ ]:
ig.to_csv('/content/drive/MyDrive/satria_data_penyisihan_2/ig_withoutstemming_label.csv', index = False)

## Data Dulu

In [ ]:
ig=pd.read_csv('/content/drive/MyDrive/Eksperimen/ig_sentiment_done.csv') #bekas dulu nlp
freq_tab.to_csv('/content/drive/MyDrive/Eksperimen/freq_tab.csv', index = False) #data dulu

## Perbandingan data 2022 dan 2021